In [2]:
# Internal modules
import itertools
import math

# External modules
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from tqdm import tqdm, trange

# Own modules
import os, sys
sys.path.append(f'{os.getcwd()}/../')

from src.environment import TrafficModel, Car, create_cars, build_network
from src.analysis import compute_regression, analyze_fairness
from src.util import change_value_of_money

plt.rcParams['text.usetex'] = True

In [6]:
def create_multiple_braess_network(k=0, capacity=100):
    network = nx.DiGraph(
        [('s0', 'v'), ('s0', 'w'), ('v', 'w'), ('v', 't0'), ('w', 't0')]
        + [(f's{i}', f's{i-1}') for i in range(1, k + 1)]
        + [(f's{i}', 'w') for i in range(1, k + 1)]
        + [(f's{i-1}', f't{i}') for i in range(1, k + 1)]
        + [('w', f't{i}') for i in range(1, k + 1)]
    )

    angle = 2 * math.pi / (2 * k + 4)
    nx.set_node_attributes(
        network,
        {
            **{'v': (3 * math.cos(-1 * angle), 3 * math.sin(-1 * angle)), 'w': (0, 0)},
            **{f's{i}': (3 * math.cos((2*i + 1) * angle), 3 * math.sin((2*i + 1) * angle)) for i in range(0, k + 1)},
            **{f't{i}': (1.5 * math.cos(2*i * angle), 1.5 * math.sin(2*i * angle)) for i in range(0, k + 1)}
        },
        "position",
    )

    nx.set_edge_attributes(
        network,
        {
            ('s0', 'v'): (2, 6, capacity, 1), ('s0', 'w'): (10, 0, 1, 1), ('v', 'w'): (1, 0, 1, 1), ('v', 't0'): (10, 0, 1, 1), ('w', 't0'): (2, 6, capacity, 1),
            **{(f's{i}', f's{i-1}'): (2, 6, capacity, 1) for i in range(1, k + 1)},
            **{(f's{i}', 'w'): (11 + 2*i, 0, 1, 1) for i in range(1, k + 1)},
            **{(f's{i-1}', f't{i}'): (11 + 2*i, 0, 1, 1) for i in range(1, k + 1)},
            **{('w', f't{i}'): (2, 6, capacity, 1) for i in range(1, k + 1)}
        },
        "latency_params",
    )

    return build_network(network)

In [7]:
def subsets(s):
    s = list(s)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s) + 1))

## For given $k$ and demand, find optimal road closure

In [26]:
k = 4
demand = k
capacity = 20
number_of_steps = 1_000
car_counts = {(f's{demand}', f't{demand}'): 50}

results = []
for restricted_edges in tqdm(subsets(range(-1, k + 1)), total=2 ** (k + 2)):
    # Restricted
    network = create_multiple_braess_network(k=k, capacity=capacity)
    model = TrafficModel(network, create_cars(network, car_counts=car_counts))

    # Close restricted edges
    for i in restricted_edges:
        model.set_edge_restriction((f's{i}' if i >= 0 else 'v', 'w'), False)

    step_stats, car_stats = model.run_sequentially(number_of_steps, show_progress=False)
    results.append({'restricted_edges': restricted_edges, 'travel_time': car_stats["travel_time"].mean()})

results = pd.DataFrame(results)

100%|██████████| 32/32 [00:33<00:00,  1.04s/it]


In [27]:
results.sort_values('travel_time')

,restricted_edges,travel_time
26,"(-1, 0, 1, 2)",20.709604
16,"(-1, 0, 1)",20.709604
6,"(-1, 0)",20.824667
17,"(-1, 0, 2)",20.824667
1,"(-1,)",20.880490
7,"(-1, 1)",20.880490
8,"(-1, 2)",20.880490
19,"(-1, 1, 2)",20.880490
31,"(-1, 0, 1, 2, 3)",22.872672
27,"(-1, 0, 1, 3)",22.872672


## For all $k$ and demands $d$, check if closing $\{-1, 0, ..., d - 1\}$ is optimal

In [86]:
k_max = 4
capacity = 20
number_of_steps = 1_000

results = []
for k in range(k_max + 1):
    for d in range(k + 1):
        car_counts = {(f's{d}', f't{d}'): 50}

        result = []
        for restricted_edges in tqdm(subsets(range(-1, k + 1)), total=2 ** (k + 2)):
            # Restricted
            network = create_multiple_braess_network(k=k, capacity=capacity)
            model = TrafficModel(network, create_cars(network, car_counts=car_counts))

            # Close restricted edges
            for i in restricted_edges:
                model.set_edge_restriction((f's{i}' if i >= 0 else 'v', 'w'), False)

            step_stats, car_stats = model.run_sequentially(number_of_steps, show_progress=False)
            result.append({'restricted_edges': restricted_edges, 'travel_time': car_stats["travel_time"].mean()})

        result = pd.DataFrame(result).set_index('restricted_edges').rename_axis(None)
        
        results.append({'k': k, 'demand': d, 'check': result.loc[[tuple(range(-1, d))]].iloc[0]['travel_time'] == result['travel_time'].min()})

results = pd.DataFrame(results)

100%|██████████| 64/64 [01:14<00:00,  1.16s/it]


In [87]:
results

,k,demand,check
0,0,0,True
1,1,0,True
2,1,1,True
3,2,0,True
4,2,1,True
5,2,2,True
6,3,0,True
7,3,1,True
8,3,2,True
9,3,3,True
